# Data preparation
Í þessu reikniriti er tekið við gögnum eftir fyrstu samhæfinu og unnin frekar.

In [3]:
from collections import defaultdict, Counter, OrderedDict
import os
import pathlib
from pathlib import Path
import re
from pprint import pprint
import importlib
from typing import List, Sequence

import matplotlib.pyplot as plt
import numpy as np

import frontend.core as c
import frontend.bulk as b
import frontend.definitions as d

%matplotlib notebook
c.THREADS = 10
c.MEMORY = 17179869184 # 16GB

working_dir = pathlib.Path('/work/haukurpj')
data_dir = working_dir.joinpath('data')
processing_dir = working_dir.joinpath('process')
p = processing_dir
parice_dir = data_dir.joinpath('parice')
rmh_dir = data_dir.joinpath('risamalheild')

IS = c.Lang.IS
EN = c.Lang.EN

RMH, PARICE = 'rmh', 'parice'
TRAIN, VAL, TEST = 'train', 'val', 'test'

langs = [IS, EN]
splits = [TRAIN, VAL, TEST]

SENT_FIX = 'sent_fix'
PROCESSED = 'processed'
TOK_LOW = 'tok_low'

FINAL = 'final'

In [8]:
{**d.SUB_URI, **{'flags': re.IGNORECASE}}

{'pattern': re.compile(r'((http(s)?:\/\/)|(www)|([-a-zA-Z0-9:%_\+.~#?&/=]+?@))+([-a-zA-Z0-9@:%_\+.~#?&/=]+)',
 re.UNICODE),
 'repl': '_uri_',
 'flags': <RegexFlag.IGNORECASE: 2>}

In [17]:
from functools import partial

URI = re.compile(r"((http(s)?:\/\/)|(www)|([-a-zA-Z0-9:%_\+.~#?&/=]+?@))+([-a-zA-Z0-9@:%_\+.~#?&/=]+)", re.IGNORECASE)
URI_SIMPLE = re.compile(r"([-a-zA-Z0-9@:%_\+.~#?&/=]+?)(\.is|\.com)", re.IGNORECASE)

def preprocess_sent(sent, lang, method):
    # print(sent)
    
    regexps = [
        {
            'pattern': URI,
            'repl': '_uri_'
        },
        {
            'pattern': URI_SIMPLE,
            'repl': '_uri_'
        },
        d.SUB_EMPTY_BRACKETS,
        {
            'pattern': re.compile(r"(\d+(.\d+)?)(mgr|gr|skv|og|eða|til|með|janúar|febrúar|mars|apríl|maí|júní|júlí|ágúst|september|október|nóvember|desember)", re.IGNORECASE),
            'repl': r"\1. \3",
        },
        {
            'pattern': re.compile(r"(skv)(?=[^.])"),
            'repl': r"\1."
        },
        {
            'pattern': re.compile(r"(\d+(.\d+)?\. )(mgr|gr)(?=[^.])", re.IGNORECASE),
            'repl': r"\1\3. "
        },
    ]
    sent = c.regexp(sent, regexps)
    sent = c.tokenize(sent, lang, method=method)
    sent = c.lowercase_normalize(sent)
    regexps = [
        d.SUB_PIPE,
        d.SUB_LT,
        d.SUB_GT,
        d.SUB_BRACKET_OPEN,
        d.SUB_BRACKET_CLOSE,
        d.SUB_FIX_PLACEHOLDERS
    ]
    sent = c.regexp(sent, regexps)
    return sent

def bulk_preprocess_sent(p_in, p_out):
    lang = b._lang(p_in)
    if lang == IS:
        method = "shallow"
    else:
        method = "moses"
    b.in_parallel(p_in, 
                  p_out,
                  c.THREADS,
                  partial(preprocess_sent, lang=lang, method=method),
                  chunksize = 10000)

In [ ]:
bulk_preprocess_sent(b.read(p, IS, PARICE, TRAIN), b.write(p, IS, PARICE, TRAIN, PROCESSED, TOK_LOW))
bulk_preprocess_sent(b.read(p, EN, PARICE, TRAIN), b.write(p, EN, PARICE, TRAIN, PROCESSED, TOK_LOW))
bulk_preprocess_sent(b.read(p, IS, PARICE, VAL), b.write(p, IS, PARICE, VAL, FINAL, TOK_LOW))
bulk_preprocess_sent(b.read(p, EN, PARICE, VAL), b.write(p, EN, PARICE, VAL, FINAL, TOK_LOW))
bulk_preprocess_sent(b.read(p, IS, PARICE, TEST), b.write(p, IS, PARICE, TEST, FINAL, TOK_LOW))
bulk_preprocess_sent(b.read(p, EN, PARICE, TEST), b.write(p, EN, PARICE, TEST, FINAL, TOK_LOW))
bulk_preprocess_sent(b.read(p, IS, RMH, SENT_FIX), b.write(p, IS, RMH, FINAL, TOK_LOW))
bulk_preprocess_sent(b.read(p, EN, 'mono'), b.write(p, EN, 'mono', FINAL, TOK_LOW))







  0%|          | 0/3540825 [00:00<?, ?it/s]




 29%|██▉       | 1044001/3540825 [00:59<01:23, 30022.51it/s]





  0%|          | 1/3540825 [00:03<3597:35:51,  3.66s/it]





  0%|          | 13418/3540825 [00:03<2508:46:40,  2.56s/it]





  1%|          | 28342/3540825 [00:03<1748:42:59,  1.79s/it]





  1%|▏         | 46503/3540825 [00:03<1217:46:27,  1.25s/it]





  2%|▏         | 62922/3540825 [00:04<848:26:19,  1.14it/s] 





  2%|▏         | 80155/3540825 [00:04<590:57:57,  1.63it/s]





  3%|▎         | 93372/3540825 [00:04<412:05:54,  2.32it/s]





  3%|▎         | 106465/3540825 [00:07<287:25:57,  3.32it/s]





  3%|▎         | 120001/3540825 [00:07<200:24:51,  4.74it/s]





  4%|▍         | 147099/3540825 [00:07<139:10:46,  6.77it/s]





  5%|▌         | 180001/3540825 [00:07<96:28:56,  9.68it/s] 





  6%|▌         | 200001/3540825 [00:10<67:10:52, 13.81it/s]





  6%|▌         | 219044/3540825 [00:10<46:45:37, 19.73it/s]





  7%|▋         | 239819/354082

 42%|████▏     | 1499091/3540825 [00:54<00:43, 46546.16it/s]





 43%|████▎     | 1509532/3540825 [00:56<02:04, 16262.46it/s]





 43%|████▎     | 1517034/3540825 [00:56<01:35, 21225.21it/s]





 43%|████▎     | 1524497/3540825 [00:56<01:38, 20543.46it/s]





 43%|████▎     | 1530295/3540825 [00:57<02:09, 15496.68it/s]





 44%|████▎     | 1546724/3540825 [00:57<01:33, 21250.64it/s]





 44%|████▍     | 1554543/3540825 [00:57<01:16, 25803.06it/s]





 44%|████▍     | 1561564/3540825 [00:57<01:03, 31139.36it/s]





 45%|████▍     | 1579930/3540825 [00:57<00:47, 41455.21it/s]





 45%|████▍     | 1590113/3540825 [00:58<00:55, 34971.10it/s]





 45%|████▌     | 1600001/3540825 [00:59<02:02, 15903.81it/s]





 45%|████▌     | 1610001/3540825 [00:59<01:32, 20882.07it/s]





 46%|████▌     | 1620001/3540825 [01:00<01:37, 19794.55it/s]





 46%|████▌     | 1630001/3540825 [01:00<01:32, 20753.14it/s]





 46%|████▋     | 1640001/3540825 [01:00<01:10, 27150.16it/s]





 47%|████▋

 71%|███████▏  | 2530001/3540825 [01:32<00:56, 17997.66it/s]





 72%|███████▏  | 2540001/3540825 [01:32<00:42, 23368.77it/s]





 72%|███████▏  | 2550001/3540825 [01:32<00:34, 28510.62it/s]





 73%|███████▎  | 2570001/3540825 [01:32<00:25, 38280.72it/s]





 73%|███████▎  | 2580001/3540825 [01:32<00:26, 36209.47it/s]





 73%|███████▎  | 2590001/3540825 [01:33<00:26, 36397.18it/s]





 73%|███████▎  | 2600001/3540825 [01:34<00:47, 19794.48it/s]





 74%|███████▍  | 2614611/3540825 [01:34<00:34, 26725.95it/s]





 74%|███████▍  | 2622298/3540825 [01:34<00:44, 20492.36it/s]





 74%|███████▍  | 2630001/3540825 [01:35<00:57, 15773.68it/s]





 75%|███████▍  | 2640001/3540825 [01:35<00:44, 20292.56it/s]





 75%|███████▍  | 2655419/3540825 [01:35<00:32, 27441.45it/s]





 75%|███████▌  | 2663463/3540825 [01:36<00:33, 26253.20it/s]





 76%|███████▌  | 2679969/3540825 [01:36<00:24, 35111.19it/s]





 76%|███████▌  | 2690001/3540825 [01:36<00:23, 35798.12it/s]





 76%|█████

 10%|█         | 362777/3540825 [00:23<82:39:45, 10.68it/s] 





 11%|█         | 389229/3540825 [00:23<57:22:59, 15.26it/s]





 12%|█▏        | 413393/3540825 [00:28<39:54:56, 21.76it/s]





 12%|█▏        | 440001/3540825 [00:28<27:42:15, 31.09it/s]





 13%|█▎        | 458996/3540825 [00:28<19:16:32, 44.41it/s]





 14%|█▎        | 479120/3540825 [00:28<13:24:22, 63.44it/s]





 14%|█▍        | 500001/3540825 [00:33<9:23:04, 90.01it/s] 





 15%|█▌        | 537745/3540825 [00:34<6:29:17, 128.57it/s]





 16%|█▌        | 558291/3540825 [00:34<4:30:46, 183.58it/s]





 17%|█▋        | 591073/3540825 [00:34<3:07:30, 262.19it/s]





 17%|█▋        | 613313/3540825 [00:39<2:13:35, 365.23it/s]





 18%|█▊        | 640001/3540825 [00:39<1:32:43, 521.39it/s]





 19%|█▊        | 657589/3540825 [00:39<1:04:37, 743.60it/s]





 19%|█▉        | 690361/3540825 [00:39<44:45, 1061.25it/s] 





 20%|██        | 711612/3540825 [00:44<34:25, 1369.45it/s]





 21%|██        | 730001/3

 66%|██████▋   | 2352089/3540825 [02:10<01:00, 19735.18it/s]





 67%|██████▋   | 2378162/3540825 [02:10<00:42, 27307.27it/s]





 68%|██████▊   | 2394105/3540825 [02:10<00:32, 35202.66it/s]





 68%|██████▊   | 2408511/3540825 [02:15<02:08, 8782.53it/s] 





 68%|██████▊   | 2420001/3540825 [02:15<01:35, 11735.01it/s]





 69%|██████▊   | 2430001/3540825 [02:15<01:14, 15004.71it/s]





 69%|██████▉   | 2454479/3540825 [02:16<00:52, 20886.59it/s]





 70%|██████▉   | 2470601/3540825 [02:16<00:37, 28248.89it/s]





 70%|███████   | 2490001/3540825 [02:16<00:29, 35747.58it/s]





 71%|███████   | 2502303/3540825 [02:20<02:11, 7878.71it/s] 





 71%|███████   | 2520001/3540825 [02:20<01:35, 10706.91it/s]





 71%|███████▏  | 2530001/3540825 [02:21<01:15, 13464.39it/s]





 72%|███████▏  | 2542017/3540825 [02:21<00:54, 18353.45it/s]





 73%|███████▎  | 2583978/3540825 [02:21<00:37, 25736.76it/s]





 73%|███████▎  | 2602352/3540825 [02:26<01:35, 9847.57it/s] 





 74%|█████

...

 24%|██▎       | 28930001/121939848 [29:18<1:31:16, 16982.31it/s]





 24%|██▎       | 28940001/121939848 [29:18<1:11:02, 21817.54it/s]





 24%|██▎       | 28950001/121939848 [29:18<54:35, 28390.16it/s]  





 24%|██▎       | 28960001/121939848 [29:18<47:10, 32845.72it/s]





 24%|██▍       | 28970001/121939848 [29:19<1:15:52, 20420.09it/s]





 24%|██▍       | 28980001/121939848 [29:21<2:06:38, 12233.77it/s]





 24%|██▍       | 28990001/121939848 [29:22<2:08:19, 12071.47it/s]





 24%|██▍       | 29003578/121939848 [29:22<1:33:14, 16611.96it/s]





 24%|██▍       | 29010001/121939848 [29:23<2:35:11, 9979.70it/s] 





 24%|██▍       | 29020001/121939848 [29:24<2:25:52, 10615.93it/s]





 24%|██▍       | 29030001/121939848 [29:24<1:48:37, 14254.90it/s]





 24%|██▍       | 29040001/121939848 [29:24<1:20:40, 19190.78it/s]





 24%|██▍       | 29050001/121939848 [29:24<1:04:43, 23918.45it/s]





 24%|██▍       | 29060001/121939848 [29:25<57:18, 27011.17it/s]  





 24%|██▍

 25%|██▍       | 29990001/121939848 [30:23<2:05:20, 12226.73it/s]





 25%|██▍       | 30010001/121939848 [30:24<1:57:19, 13059.37it/s]





 25%|██▍       | 30020001/121939848 [30:25<2:13:06, 11509.97it/s]





 25%|██▍       | 30030001/121939848 [30:25<1:41:50, 15041.04it/s]





 25%|██▍       | 30040001/121939848 [30:26<1:24:39, 18091.87it/s]





 25%|██▍       | 30059575/121939848 [30:26<1:01:41, 24819.20it/s]





 25%|██▍       | 30070001/121939848 [30:27<1:31:36, 16715.22it/s]





 25%|██▍       | 30080001/121939848 [30:28<2:13:05, 11502.84it/s]





 25%|██▍       | 30090001/121939848 [30:29<1:56:05, 13186.44it/s]





 25%|██▍       | 30110001/121939848 [30:30<1:52:30, 13603.32it/s]





 25%|██▍       | 30120001/121939848 [30:32<2:13:00, 11504.81it/s]





 25%|██▍       | 30132465/121939848 [30:32<1:36:46, 15810.00it/s]





 25%|██▍       | 30140001/121939848 [30:32<1:33:26, 16374.39it/s]





 25%|██▍       | 30170001/121939848 [30:33<1:21:06, 18856.67it/s]





 25%|█

 25%|██▌       | 31070001/121939848 [31:28<1:33:28, 16202.62it/s]





 25%|██▌       | 31080001/121939848 [31:30<2:08:29, 11785.51it/s]





 25%|██▌       | 31090001/121939848 [31:30<1:48:29, 13957.52it/s]





 26%|██▌       | 31110001/121939848 [31:31<1:44:07, 14537.74it/s]





 26%|██▌       | 31120001/121939848 [31:32<1:51:31, 13571.82it/s]





 26%|██▌       | 31130001/121939848 [31:33<1:57:51, 12842.50it/s]





 26%|██▌       | 31150001/121939848 [31:33<1:27:05, 17373.74it/s]





 26%|██▌       | 31167368/121939848 [31:33<1:03:34, 23799.24it/s]





 26%|██▌       | 31175589/121939848 [31:35<1:47:31, 14069.14it/s]





 26%|██▌       | 31181560/121939848 [31:36<2:33:48, 9834.75it/s] 





 26%|██▌       | 31190001/121939848 [31:36<2:14:44, 11224.75it/s]





 26%|██▌       | 31200001/121939848 [31:36<1:45:27, 14341.55it/s]





 26%|██▌       | 31210001/121939848 [31:37<2:06:41, 11936.15it/s]





 26%|██▌       | 31220001/121939848 [31:38<2:05:58, 12002.90it/s]





 26%|█

 26%|██▋       | 32090001/121939848 [32:31<1:52:33, 13304.41it/s]





 26%|██▋       | 32100001/121939848 [32:31<1:31:54, 16291.87it/s]





 26%|██▋       | 32110001/121939848 [32:33<1:57:25, 12750.61it/s]





 26%|██▋       | 32120001/121939848 [32:34<2:17:22, 10897.21it/s]





 26%|██▋       | 32140001/121939848 [32:34<1:47:56, 13864.87it/s]





 26%|██▋       | 32160001/121939848 [32:35<1:23:01, 18023.06it/s]





 26%|██▋       | 32170001/121939848 [32:36<1:40:35, 14873.10it/s]





 26%|██▋       | 32180001/121939848 [32:37<1:50:38, 13521.83it/s]





 26%|██▋       | 32190001/121939848 [32:37<1:50:09, 13579.64it/s]





 26%|██▋       | 32200001/121939848 [32:37<1:26:23, 17311.67it/s]





 26%|██▋       | 32210001/121939848 [32:39<1:58:57, 12570.89it/s]





 26%|██▋       | 32220001/121939848 [32:40<2:17:13, 10896.34it/s]





 26%|██▋       | 32240001/121939848 [32:40<1:45:53, 14117.80it/s]





 26%|██▋       | 32260001/121939848 [32:41<1:20:09, 18645.30it/s]





 26%|█

 27%|██▋       | 33170001/121939848 [33:37<1:48:30, 13634.06it/s]





 27%|██▋       | 33180001/121939848 [33:38<1:41:43, 14542.87it/s]





 27%|██▋       | 33190001/121939848 [33:38<1:37:03, 15238.82it/s]





 27%|██▋       | 33204999/121939848 [33:38<1:10:53, 20861.30it/s]





 27%|██▋       | 33210957/121939848 [33:40<3:07:47, 7874.67it/s] 





 27%|██▋       | 33220001/121939848 [33:41<3:02:34, 8099.03it/s]





 27%|██▋       | 33250001/121939848 [33:42<2:12:40, 11141.20it/s]





 27%|██▋       | 33260001/121939848 [33:42<2:06:29, 11684.81it/s]





 27%|██▋       | 33270001/121939848 [33:43<2:06:38, 11670.06it/s]





 27%|██▋       | 33280001/121939848 [33:44<1:52:06, 13180.50it/s]





 27%|██▋       | 33290001/121939848 [33:44<1:46:58, 13811.11it/s]





 27%|██▋       | 33310001/121939848 [33:46<1:55:20, 12806.58it/s]





 27%|██▋       | 33320001/121939848 [33:47<2:12:55, 11111.47it/s]





 27%|██▋       | 33350001/121939848 [33:48<1:37:55, 15078.66it/s]





 27%|██

We are now done processing the val, test, EN mono and RMH datasets.

In [10]:
!head {b.read(p, IS, PARICE, TRAIN, PROCESSED, TOK_LOW)}

í slíkum tilvikum má heimila minni austursbrunna sem geta tekið við afrennslinu á hæfilega löngum tíma .
• hættuflokka 3.1 til 3.6 , 3.7 , skaðleg áhrif á kynstarfsemi og frjósemi eða þroskun , 3.8 , áhrif önnur en sljóleiki , 3.9 og 3.10 ,
rannsóknir á eiturverkunum eftir endurtekna skammta leiddu í ljós frymisbólumyndun á nýrnapíplum og bentu rannsóknir sterklega til að þessi áhrif væru afturkallanleg .
samtímismeðferð með meðalmiklum cyp 3 a 4 hemlum ( t.d. dílítazem , verapamíl , klarítrómýcín , etýtrómýcín , aprepítant , amíódarón ) skal aðeins gefin með varúð hjá sjúklingum sem fá 25 mg og hana á að forðast hjá sjúklingum sem fá temsirolimus í stærri skömmtum en 25 mg .
gæta á varúðar við notkun lyfsins hjá sjúklingum með brátt kransæðaheilkenni :
• afkastageta ,
í skóla dr. augustine ?
leifamerki
sítrónur
hnúðsilla


## Fjarlægja slæmar línur
Núna búum við til safn af "íslenskum" orðum frá RMH og förum yfir íslenskar setningar í ParIce og athugum hversu stórt hlutfall af orðunum í ParIce-IS eru í safninu okkar.

In [30]:
is_counter_1 = b.token_counter(b.read(p, IS, PARICE, TRAIN, PROCESSED))
print(len(is_counter_1))

655511


In [12]:
rmh_counter = b.token_counter(b.read(p, IS, RMH, PROCESSED))
is_words = set(rmh_counter.keys())
len(is_words)

6073000

In [14]:
# Setting the chunksize higher is better here.
print(b.CHUNKSIZE)
b.CHUNKSIZE = 50000
print(b.CHUNKSIZE)

4000
50000


In [25]:
skip_lines = b.get_drop_lines(b.read(p, IS, PARICE, TRAIN, PROCESSED),
                              [c.REGEXP_SUB['CRYLLIC'][0],
                               c.REGEXP_SUB['GREEK'][0],
                               c.REGEXP_SUB['UNKNOWN-CHARS'][0]
                              ],
                              is_words,
                              keep_ratio=0.8,
                              normalize=True,
                              keep_sent_length=1)


100%|██████████| 3540825/3540825 [02:30<00:00, 23545.40it/s] 


In [26]:
lines = [number for number, fraction, line in skip_lines]
print("new fraction", (1 - (b.info(b.read(p, IS, PARICE, TRAIN, PROCESSED))[2] - len(lines))/b.info(b.read(p, IS, PARICE, TRAIN, PROCESSED))[2]))

new fraction 0.04822237755325387


In [28]:
b.drop_lines(b.read(p, IS, PARICE, TRAIN, PROCESSED),
             b.write(p, IS, PARICE, TRAIN, FINAL),
             lines_in=lines)
b.drop_lines(b.read(p, EN, PARICE, TRAIN, PROCESSED),
             b.write(p, EN, PARICE, TRAIN, FINAL),
             lines_in=lines)

True

In [32]:
is_counter_2 = b.token_counter(b.read(p, IS, PARICE, TRAIN, FINAL))
print(len(is_counter_2))
print(1-(len(is_counter_2)/(len(is_counter_1))))

580166
0.11494086292983641
